In [1]:
import numpy as np
import constriction

In [2]:
np.random.seed(202404151)
matrix = np.random.randn(1024, 1024)
matrix

array([[-0.24281341,  0.82277433, -1.76817447, ...,  0.37827248,
         0.95446806,  1.3724337 ],
       [-0.93972788,  0.53986172,  0.49766536, ...,  1.92457392,
        -0.32346853,  2.16728419],
       [ 0.77583578, -1.37499597, -0.03150158, ..., -1.95524463,
         0.77135577,  0.56992074],
       ...,
       [-1.11369009, -0.3860364 , -0.74349466, ...,  1.20332443,
         0.25760983,  1.11943256],
       [-0.73039511, -0.79896008, -0.80715964, ...,  0.54681539,
        -0.4218426 , -0.47858875],
       [ 0.11453367, -0.5499435 ,  0.71707023, ...,  0.45978248,
         0.60778436, -0.13992389]])

In [3]:
quantized_matrix = np.round(matrix * 8).astype(np.int8)
print(f'min: {quantized_matrix.min()}; max: {quantized_matrix.max()}')
quantized_matrix

min: -41; max: 38


array([[ -2,   7, -14, ...,   3,   8,  11],
       [ -8,   4,   4, ...,  15,  -3,  17],
       [  6, -11,   0, ..., -16,   6,   5],
       ...,
       [ -9,  -3,  -6, ...,  10,   2,   9],
       [ -6,  -6,  -6, ...,   4,  -3,  -4],
       [  1,  -4,   6, ...,   4,   5,  -1]], dtype=int8)

In [4]:
np.unique(quantized_matrix.ravel(), return_counts=True)

(array([-41, -38, -36, -35, -34, -33, -32, -31, -30, -29, -28, -27, -26,
        -25, -24, -23, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13,
        -12, -11, -10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,
          1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  38], dtype=int8),
 array([    1,     2,     3,     3,     6,     9,    20,    40,    36,
           73,   108,   187,   272,   409,   564,   862,  1159,  1656,
         2296,  3194,  4182,  5575,  7191,  9100, 11442, 13800, 17154,
        20278, 23604, 27970, 31746, 35848, 38960, 43094, 46095, 48689,
        50576, 51960, 52311, 51639, 50460, 48842, 46323, 43185, 39306,
        35732, 31837, 27474, 23987, 20475, 16945, 13971, 11278,  9067,
         7086,  5327,  4310,  3063,  2328,  1626,  1214,   853,   601,
          421,   254,   174,   122,    75,    54,    27,    2

In [5]:
values, counts = np.unique(quantized_matrix.ravel(), return_counts=True)
order = np.argsort(counts)[::-1]
values = values[order]
counts = counts[order]
entropy20bit = np.log2(matrix.size) - counts @ np.log2(counts) / matrix.size
entropy20bit

5.048731287507302

In [6]:
freqs = np.maximum(np.round(counts / 256).astype(np.int64), 1)
too_much = sum(freqs) - 2**12
print(too_much, 1024**2 - sum(counts))
freqs[:too_much] -= 1 # very hacky but good enough for now
print(2**12 - sum(freqs), 1024**2 - sum(counts))
freqs

19 0
0 0


array([203, 202, 201, 197, 196, 190, 189, 180, 179, 168, 167, 153, 151,
       139, 139, 123, 123, 108, 106,  94,  92,  80,  79,  67,  66,  55,
        54,  45,  44,  36,  35,  28,  28,  22,  21,  17,  16,  12,  12,
         9,   9,   6,   6,   5,   5,   3,   3,   2,   2,   2,   2,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1])

In [7]:
cross_entropy12bit = 12 - counts @ np.log2(freqs) / (1024**2)
cross_entropy12bit, entropy20bit

(5.054123235441871, 5.048731287507302)

In [8]:
cdf = np.add.accumulate(np.concatenate([np.array([0]), freqs]))
values_pad = np.concatenate([values, np.array([0])])
model_specification = np.stack((cdf, values_pad), axis=1).astype(np.int16).ravel()[:-1]
model_specification, len(model_specification), 2 * len(freqs), 1024**2

(array([   0,    0,  203,   -1,  405,    1,  606,   -2,  803,    2,  999,
           3, 1189,   -3, 1378,    4, 1558,   -4, 1737,    5, 1905,   -5,
        2072,    6, 2225,   -6, 2376,   -7, 2515,    7, 2654,    8, 2777,
          -8, 2900,   -9, 3008,    9, 3114,   10, 3208,  -10, 3300,   11,
        3380,  -11, 3459,  -12, 3526,   12, 3592,   13, 3647,  -13, 3701,
         -14, 3746,   14, 3790,  -15, 3826,   15, 3861,  -16, 3889,   16,
        3917,  -17, 3939,   17, 3960,   18, 3977,  -18, 3993,  -19, 4005,
          19, 4017,   20, 4026,  -20, 4035,  -21, 4041,   21, 4047,   22,
        4052,  -22, 4057,  -23, 4060,   23, 4063,   24, 4065,  -24, 4067,
          25, 4069,  -25, 4071,  -26, 4072,   26, 4073,  -27, 4074,   27,
        4075,   28, 4076,  -28, 4077,   29, 4078,  -29, 4079,   30, 4080,
         -31, 4081,  -30, 4082,   31, 4083,   32, 4084,  -32, 4085,   33,
        4086,  -33, 4087,  -34, 4088,   34, 4089,   35, 4090,  -35, 4091,
         -36, 4092,  -38, 4093,   36, 

In [9]:
inv_vocab = {value: i for i, value in enumerate(values)}

In [164]:
class AnsCoder:
    def __init__(self, precision, word_size, compressed=[]):
        self.precision = precision
        self.word_size = word_size
        self.word_mask = (1 << word_size) - 1 # (a string of `word_size`` one-bits)
        self.quantile_mask = (1 << precision) - 1 # (a string of `precision`` one-bits)
        self.bulk = compressed.copy() # (We will mutate `bulk`` below.)
        self.head = 0
        # Establish invariant (ii):
        while len(self.bulk) != 0 and (self.head >> word_size) == 0:
            self.head = (self.head << word_size) | self.bulk.pop()

    def push(self, symbol, m): # Encodes one symbol.
        # Check if encoding directly onto head would violate invariant (i):
        if (self.head >> (2 * self.word_size - self.precision)) >= m[symbol]:
            # Transfer one word of compressed data from head to bulk:
            self.bulk.append(self.head & self.word_mask) # (“&” is bitwise `and`)
            self.head >>= self.word_size
            # At this point, invariant (ii) is definitely violated,
            # but the operations below will restore it.

        z = self.head % m[symbol] + sum(m[0:symbol])
        self.head //= m[symbol]
        self.head = (self.head << self.precision) | z # (This is
            # equivalent to “ self.head * n + z”, just slightly faster.)

    def pop(self, m): # Decodes one symbol.
        z = self.head & self.quantile_mask # (same as “ self.head % n” but faster)
        self.head >>= self.precision # (same as “ //= n” but faster)
        for symbol, m_symbol in enumerate(m):
            if z >= m_symbol:
                z -= m_symbol
            else:
                break # We found the symbol that satisfies z ∈ Z_i(symbol).
        self.head = self.head * m_symbol + z
        print(f'm_symbol = {m_symbol}')
        print(f'quantile = {z}')

        # Restore invariant (ii) if it is violated (which happens exactly
        # if the encoder transferred data from head to bulk at this point):
        if (self.head >> self.word_size) == 0 and len(self.bulk) != 0:
            # Transfer data back from bulk to head (“ |” is bitwise or):
            self.head = (self.head << self.word_size) | self.bulk.pop()

        return symbol

    def get_compressed(self):
        compressed = self.bulk.copy() # (We will mutate `compressed`` below.)
        head = self.head
        # Chop `head`` into `word_size`-sized words and append to `compressed``:
        while head != 0:
            compressed.append(head & self.word_mask)
            head >>= self.word_size
        return compressed

In [11]:
distribution = constriction.stream.model.Categorical(freqs.astype(np.float64))
# fake_next_data = np.array([1, 1], dtype=np.uint32) # pretend that there's more data to come (for the next matrix) so that decoding doesn't interfere with next row
def compress_row(row):
    row = np.array([inv_vocab[value] for value in row], dtype=np.int32)
    # coder = constriction.stream.stack.AnsCoder(fake_next_data)
    coder = AnsCoder(12, 16, [1, 1])
    for elem in row[::-1]:
        coder.push(elem, freqs)
    # coder.encode_reverse(row.astype(np.int32), distribution)
    return np.array(coder.get_compressed(), dtype=np.uint16)[::-1]
    # return coder.get_compressed()

In [12]:
compressed = [compress_row(row) for row in quantized_matrix]

In [13]:
len(compressed[0]) * 16 / 1024, cross_entropy12bit

(5.109375, 5.054123235441871)

In [84]:
offsets = np.zeros((2 * 1024,), dtype=np.uint16)
pos = len(model_specification) + len(offsets)
mask = (1 << 16) - 1
for i in range(1024):
    # little endian encoding
    offsets[2 * i] = pos & mask
    offsets[2 * i + 1] = pos >> 16
    pos += len(compressed[i])
offsets, len(offsets)

(array([2201,    0, 2528, ...,    5, 6956,    5], dtype=uint16), 2048)

In [85]:
serialized = np.concatenate([offsets, model_specification.astype(np.uint16)] + compressed)

In [86]:
len(offsets) / 256

8.0

In [87]:
[serialized[i] for i in offsets]

[20827,
 2201,
 273,
 2201,
 3819,
 2201,
 1188,
 2201,
 2,
 2201,
 1935,
 2201,
 11,
 2201,
 4584,
 2201,
 46,
 2201,
 192,
 2201,
 11852,
 2201,
 88,
 2201,
 2,
 2201,
 6161,
 2201,
 1,
 2201,
 61,
 2201,
 129,
 2201,
 83,
 2201,
 77,
 2201,
 21074,
 2201,
 145,
 2201,
 1504,
 2201,
 990,
 2201,
 48,
 2201,
 15750,
 2201,
 2074,
 2201,
 5,
 2201,
 116,
 2201,
 32,
 2201,
 4109,
 2201,
 630,
 2201,
 260,
 2201,
 6,
 2201,
 31833,
 2201,
 684,
 2201,
 34983,
 2201,
 3,
 2201,
 12,
 2201,
 1052,
 2201,
 12148,
 2201,
 24,
 2201,
 714,
 2201,
 46,
 2201,
 2,
 2201,
 42,
 2201,
 10,
 2201,
 8,
 2201,
 16,
 2201,
 1,
 2201,
 749,
 2201,
 20962,
 2201,
 19080,
 2201,
 4698,
 2201,
 51,
 2201,
 1807,
 2201,
 54159,
 2201,
 18108,
 2201,
 9,
 2201,
 3024,
 2201,
 1044,
 2201,
 74,
 2201,
 333,
 2201,
 250,
 2201,
 378,
 2201,
 20,
 2201,
 1417,
 2201,
 30,
 2201,
 25,
 2201,
 185,
 2201,
 1311,
 2201,
 3512,
 2201,
 1897,
 2201,
 4,
 2201,
 34,
 2201,
 13,
 2201,
 1777,
 2201,
 10330,
 2201,


In [88]:
model_specification

array([   0,    0,  203,   -1,  405,    1,  606,   -2,  803,    2,  999,
          3, 1189,   -3, 1378,    4, 1558,   -4, 1737,    5, 1905,   -5,
       2072,    6, 2225,   -6, 2376,   -7, 2515,    7, 2654,    8, 2777,
         -8, 2900,   -9, 3008,    9, 3114,   10, 3208,  -10, 3300,   11,
       3380,  -11, 3459,  -12, 3526,   12, 3592,   13, 3647,  -13, 3701,
        -14, 3746,   14, 3790,  -15, 3826,   15, 3861,  -16, 3889,   16,
       3917,  -17, 3939,   17, 3960,   18, 3977,  -18, 3993,  -19, 4005,
         19, 4017,   20, 4026,  -20, 4035,  -21, 4041,   21, 4047,   22,
       4052,  -22, 4057,  -23, 4060,   23, 4063,   24, 4065,  -24, 4067,
         25, 4069,  -25, 4071,  -26, 4072,   26, 4073,  -27, 4074,   27,
       4075,   28, 4076,  -28, 4077,   29, 4078,  -29, 4079,   30, 4080,
        -31, 4081,  -30, 4082,   31, 4083,   32, 4084,  -32, 4085,   33,
       4086,  -33, 4087,  -34, 4088,   34, 4089,   35, 4090,  -35, 4091,
        -36, 4092,  -38, 4093,   36, 4094,   38, 40

In [89]:
# pad to a multiple of 512 entries for now so it's easier to read in
pad_len = (512 - len(serialized) % 512) % 512
serialized = np.concatenate([serialized, np.zeros((pad_len,), dtype=np.uint16)])

In [90]:
serialized.tofile('mock-matrix.bin')

In [91]:
serialized.shape, len(offsets), len(model_specification)

((335360,), 2048, 153)

In [92]:
!ls -l mock-matrix.bin

-rw-rw-r-- 1 robamler robamler 670720 Apr 16 19:43 mock-matrix.bin


In [93]:
670720 / 2 / 512

655.0

In [94]:
word1 = serialized[offsets[0]]
word2 = serialized[offsets[0] + 1]
quantile = (word2 % (2**24)) >> 8
print(f'quantile = {quantile}')
symbol_id = (cdf <= quantile).sum() - 1
cdf[symbol_id-1:symbol_id+2], symbol_id, values[symbol_id]

quantile = 194


(array([], dtype=int64), 0, 0)

In [95]:
quantized_matrix[0, 0]

-2

In [96]:
len(offsets) / 512

4.0

In [97]:
offsets[1::2][200:300]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=uint16)

In [98]:
offsets[::2][1000:]

array([65010, 65335,   127,   452,   774,  1098,  1421,  1748,  2076,
        2402,  2729,  3055,  3379,  3704,  4032,  4359,  4686,  5012,
        5337,  5660,  5983,  6307,  6629,  6956], dtype=uint16)

In [99]:
original_offsets = np.array([offsets[2*i] + (offsets[2*i+1] << 16) for i in range(1024)])
original_offsets

array([  2201,   2528,   2850, ..., 333987, 334309, 334636])

In [125]:
np.array([serialized[i+1] for i in original_offsets][:100])

array([49878, 31450, 43115, 33592, 18477, 37607,   617, 20837, 18140,
         777, 29705, 32476, 54692, 61970,  8413, 56750, 41112, 35941,
       59411, 29014, 51298, 34952, 29823, 27221,  6038, 16029, 56187,
       28383, 34615, 12035, 36892, 45747, 61146,  4349,  8142, 14792,
       30269, 15191,  1630, 44158, 46566, 16718, 63925, 63123,  9758,
       61283, 36444, 33762,  7186, 22353, 13345, 54457, 52447, 32507,
       12906, 25571, 11489, 24612, 18984, 19395, 59093, 14584, 50986,
         821, 19552, 42200, 50218, 54110, 10002, 41743, 33211, 47796,
       49983, 49960, 51074, 26130, 12662, 13385, 61763, 46585, 12404,
        1704, 11640, 64241, 13902, 53453, 33513, 15027, 65340,  6777,
       47872, 53306,  1701, 30464, 25630, 34004, 14576, 61048,   832,
       54095], dtype=uint16)

In [101]:
coder = AnsCoder(12, 16, list(serialized[original_offsets[0]:][::-1]))

In [102]:
np.all(serialized[original_offsets[0]:original_offsets[1]] == compressed[0])

True

In [132]:
gt_first_symbols = np.array([inv_vocab[quantized_matrix[i, 0]] for i in range(1024)])
gt_first_symbols

array([ 3, 16, 11, ..., 17, 12,  2])

In [134]:
js_first_symbols = np.array([3,16,11,4,11,3,3,1,9,3,5,29,7,2,1,23,0,19,10,1,11,11,5,14,10,27,17,29,9,30,0,3,29,1,42,13,8,17,8,19,7,1,13,8,8,34,26,4,18,9,5,6,20,30,3,4,20,0,14,18,9,12,9,4,19,6,5,4,9,3,2,15,4,4,10,7,1,5,1,7,0,8,22,16,8,1,3,15,32,15,16,0,8,9,5,6,12,27,4,4,2,15,16,11,26,6,15,2,25,15,13,8,0,21,9,17,9,22,39,3,31,20,14,25,5,15,3,30,23,23,7,18,6,26,12,3,13,5,21,35,27,2,20,11,16,8,8,43,6,6,3,12,14,33,11,14,19,41,9,8,18,12,6,9,31,0,23,30,5,0,27,1,14,1,11,11,6,22,3,1,14,40,22,5,23,18,7,7,43,2,0,9,23,40,16,44,6,2,16,12,25,4,3,11,25,28,13,14,26,15,9,12,0,23,3,22,5,10,18,13,7,26,5,9,5,5,26,8,10,4,23,5,7,26,1,11,9,9,15,21,22,26,8,10,21,16,0,9,3,2,5,29,5,21,14,3,0,7,5,2,4,2,10,13,6,16,19,28,8,33,9,14,25,2,34,8,18,11,11,9,4,17,7,12,13,18,2,2,9,13,4,10,15,16,19,20,2,0,17,3,13,7,6,18,7,8,7,16,23,0,4,9,6,3,5,10,3,11,10,11,19,5,14,19,7,29,1,2,10,2,8,18,28,3,1,7,5,20,16,14,11,2,1,11,39,16,38,34,1,9,12,18,8,7,13,19,20,9,18,7,26,2,9,1,2,30,11,11,4,29,1,6,6,26,6,22,43,9,27,7,3,7,15,30,5,16,18,11,8,33,9,6,14,0,5,14,25,9,8,16,13,21,17,38,0,22,34,4,20,14,20,5,32,14,0,8,34,12,27,5,14,4,13,35,12,25,6,5,6,12,9,3,15,19,16,10,16,6,21,16,15,22,13,2,14,30,0,12,11,4,0,3,13,17,30,29,24,3,9,12,19,2,15,17,3,26,3,10,0,12,26,24,15,10,5,14,9,27,1,20,1,4,28,1,17,2,21,1,7,5,16,3,32,4,6,5,0,27,1,29,6,21,18,0,17,2,19,14,9,9,20,8,8,1,22,20,9,1,26,4,14,9,0,38,7,9,13,0,6,0,8,14,0,15,22,0,10,0,19,11,30,21,5,9,0,3,0,26,55,13,26,13,5,5,3,21,4,0,4,3,27,4,13,9,28,7,23,10,14,6,7,23,1,4,15,21,20,8,16,25,22,4,13,2,20,18,21,0,22,5,10,0,7,11,19,3,6,14,17,6,6,1,23,6,18,21,6,11,17,20,9,47,6,20,2,3,17,4,22,9,31,9,1,11,10,4,21,7,0,5,11,9,5,6,3,9,2,18,21,7,13,11,15,13,34,15,6,0,24,19,41,33,32,10,16,5,12,3,12,1,9,4,9,7,14,4,4,22,8,41,14,0,5,8,19,14,4,10,0,29,21,22,23,18,25,1,17,6,11,15,16,10,7,22,13,3,4,2,14,13,16,5,21,18,16,10,0,12,0,5,29,11,9,11,9,21,13,27,23,21,9,7,1,18,10,10,4,19,4,19,18,42,11,18,12,14,32,1,1,1,19,13,6,3,0,12,14,23,39,4,0,6,25,24,10,24,1,10,17,17,7,8,5,10,0,23,4,14,12,26,14,13,4,1,6,23,5,20,0,2,6,9,7,6,1,7,0,5,23,17,12,25,1,7,4,6,0,10,21,18,6,9,16,17,9,0,3,17,9,5,9,6,4,6,21,6,14,6,0,11,4,9,2,7,1,6,24,17,8,24,27,21,21,9,9,3,0,6,0,31,21,21,7,11,7,15,21,19,10,7,3,15,4,7,7,4,7,13,29,3,17,4,17,14,10,19,29,7,5,5,3,1,3,20,1,18,22,7,8,18,0,3,4,17,0,11,7,30,11,13,11,12,27,9,26,2,16,28,12,12,16,24,21,6,6,3,10,5,2,6,12,8,0,13,4,2,17,23,31,5,20,14,23,6,29,1,23,4,25,30,9,4,10,8,24,0,24,1,21,6,10,18,2,11,22,12,4,1,19,12,10,1,5,3,17,5,10,2,2,8,3,24,29,24,2,18,0,3,0,14,1,2,34,11,11,18,16,4,4,10,5,19,35,3,9,25,7,7,20,0,10,22,34,10,10,5,22,5,6,11,11,25,14,11,0,9,11,2,20,4,17,9,8,6,0,0,4,2,25,5,7,17,12,2])
np.all(gt_first_symbols == js_first_symbols)

True

In [143]:
quantized_matrix[:, 0] % 2**16

array([65534, 65528,     6, ..., 65527, 65530,     1], dtype=int32)

In [146]:
quantile = [freqs[i] for i in gt_first_symbols][-10:]
quantile

[203, 203, 196, 201, 55, 190, 180, 108, 151, 201]

In [118]:
cdf

array([   0,  203,  405,  606,  803,  999, 1189, 1378, 1558, 1737, 1905,
       2072, 2225, 2376, 2515, 2654, 2777, 2900, 3008, 3114, 3208, 3300,
       3380, 3459, 3526, 3592, 3647, 3701, 3746, 3790, 3826, 3861, 3889,
       3917, 3939, 3960, 3977, 3993, 4005, 4017, 4026, 4035, 4041, 4047,
       4052, 4057, 4060, 4063, 4065, 4067, 4069, 4071, 4072, 4073, 4074,
       4075, 4076, 4077, 4078, 4079, 4080, 4081, 4082, 4083, 4084, 4085,
       4086, 4087, 4088, 4089, 4090, 4091, 4092, 4093, 4094, 4095, 4096])

In [165]:
coder = AnsCoder(12, 16, list(compressed[0][::-1]))
coder.pop(freqs)
coder.head

m_symbol = 197
quantile = [203, 203, 196, 201, 55, 190, 180, 108, 151, 201]


65649188

In [152]:
gt_first_symbols

array([ 3, 16, 11, ..., 17, 12,  2])

In [153]:
quantized_matrix[:, 1]

array([  7,   4, -11, ...,  -3,  -6,  -4], dtype=int8)

In [163]:
1364968150 % (2**12)

726

In [161]:
(1364968150 >> 12) * 197 + 726

65649794